# 04. 영어/한국어 Word2Vec 훈련시키기

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [2]:
import nltk
nltk.download('punkt')
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sswwd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x172e67236a0>)

해당 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요하다.

In [3]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [4]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## 3) Word2Vec 훈련시키기


In [6]:
# !pip install gensim

In [5]:
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)
# cbow 사용
from gensim.models import Word2Vec, KeyedVectors
model2 = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=1)
# skip-gram 사용

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.  
size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

In [8]:
model_result = model.wv.most_similar("man") #가장 유사한 단어 출력
print(model_result)

[('woman', 0.8452662825584412), ('guy', 0.7914076447486877), ('lady', 0.7580448389053345), ('boy', 0.7566693425178528), ('girl', 0.7327544093132019), ('gentleman', 0.7280296087265015), ('soldier', 0.723828911781311), ('poet', 0.6915566921234131), ('kid', 0.6702145338058472), ('surgeon', 0.6474255919456482)]


In [9]:
model_result2 = model2.wv.most_similar("man") #가장 유사한 단어 출력
print(model_result2)

[('woman', 0.7728009819984436), ('guy', 0.7634276747703552), ('boy', 0.7009593844413757), ('rabbi', 0.6902947425842285), ('titus', 0.6895440816879272), ('soldier', 0.6821202635765076), ('son', 0.6767211556434631), ('pianist', 0.6747886538505554), ('gentleman', 0.6726687550544739), ('widow', 0.6685516834259033)]


# 4) Word2Vec 모델 저장하고 로드하기

In [10]:
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [11]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8452662825584412), ('guy', 0.7914076447486877), ('lady', 0.7580448389053345), ('boy', 0.7566693425178528), ('girl', 0.7327544093132019), ('gentleman', 0.7280296087265015), ('soldier', 0.723828911781311), ('poet', 0.6915566921234131), ('kid', 0.6702145338058472), ('surgeon', 0.6474255919456482)]


# 한국어 word2vec 만들기

1. https://dumps.wikimedia.org/kowiki/latest/ 에 들어가서   
kowiki-latest-pages-articles.xml.bz2 파일 다운로드(약 40분걸림)    
2. https://github.com/attardi/wikiextractor 들어가서 code를 zip으로 다운로드   


3. 압축 풀고 cmd에서 파일 있는 경로로 가서 가상환경 설정해주고 'python setdup.py install'입력  
  
  
4. wikiextractor 폴더 안에 WikiExtractor.py 파일이 생기는데   
동일한 경로 안에 kowiki-latest-pages-articles.xml.bz2 넣어준다.  
    * A:\chchdata\data\wikiextractor-master\wikiextractor\WikiExtractor.py  
    * A:\chchdata\data\wikiextractor-master\wikiextractor\kowiki-latest-pages-articles.xml.bz2

5. WikiExtractor.py에 들어가서 파일 변경(YoungriKIM의 코드 참고)
    - from multiprocessing.dummy import Queue, Process 추가
    - 186번 줄 : return open(filename, 'w') -> return open(filename, 'w', encoding='utf-8')
    - 210번 줄 : output = open(output_file, 'w') -> output = open(output_file, 'w', encoding='utf-8')

5. cmd에 위의 경로로 들어가서 python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 입력


# 훈련데이터 만들기
### AA 디렉토리 안의 모든 파일인 wiki00 ~ wiki90에 대해서 wikiAA.txt로 통합
cmd 창에서 아래와 같이 입력  
(chch) A:\chch\chch\wikiextractor\text>  
copy A:\chch\chch\wikiextractor\text\AA\wiki* wiki1.txt   
copy A:\chch\chch\wikiextractor\text\AB\wiki* wiki2.txt  
copy A:\chch\chch\wikiextractor\text\AC\wiki* wiki3.txt  
copy A:\chch\chch\wikiextractor\text\AD\wiki* wiki4.txt  
copy A:\chch\chch\wikiextractor\text\AE\wiki* wiki5.txt  
copy A:\chch\chch\wikiextractor\text\AF\wiki* wiki6.txt  
copy A:\chch\chch\wikiextractor\text\AG\wiki* wiki7.txt  
copy A:\chch\chch\wikiextractor\text\AH\wiki* wiki8.txt  
copy A:\chch\chch\wikiextractor\text\AI\wiki* wiki9.txt  

### 위의 파일을 다시 1개의 파일로 합치기
copy A:\chch\chch\wikiextractor\text\wiki* wiki_data.txt


In [12]:
f = open('A:\\chch\\chch\\wikiextractor\\text\\wiki_data.txt', encoding="utf8")

i=0
while True:
    line = f.readline()
    if line != '\n':
        i=i+1
        print("%d번째 줄 :"%i + line)
    if i==5:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.



In [13]:
from konlpy.tag import Okt
from datetime import datetime

start = datetime.now()
okt=Okt()
fread = open('A:\\chch\\chch\\wikiextractor\\text\\wiki_data.txt', encoding="utf8")
# 파일을 다시 처음부터 읽음.
n=0
result = []

while True:
    line = fread.readline() #한 줄씩 읽음.
    if not line: break # 모두 읽으면 while문 종료.
    n=n+1
    if n%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
      result.append(temp) # 결과에 저장
fread.close()
end = datetime.now()
print('시간 : ',end-start)

5000번째 While문.
10000번째 While문.
15000번째 While문.
20000번째 While문.
25000번째 While문.
30000번째 While문.
35000번째 While문.
40000번째 While문.
45000번째 While문.
50000번째 While문.
55000번째 While문.
60000번째 While문.
65000번째 While문.
70000번째 While문.
75000번째 While문.
80000번째 While문.
85000번째 While문.
90000번째 While문.
95000번째 While문.
100000번째 While문.
105000번째 While문.
110000번째 While문.
115000번째 While문.
120000번째 While문.
125000번째 While문.
130000번째 While문.
135000번째 While문.
140000번째 While문.
145000번째 While문.
150000번째 While문.
155000번째 While문.
160000번째 While문.
165000번째 While문.
170000번째 While문.
175000번째 While문.
180000번째 While문.
185000번째 While문.


KeyboardInterrupt: 

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 1657878


# Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, vector_size=100, window=5, min_count=5, workers=4, sg=0)
model2 = Word2Vec(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model_result1=model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.6591412425041199), ('조선민주주의인민공화국', 0.5870327353477478), ('관세청', 0.5554245710372925), ('우리나라', 0.5490413904190063), ('부산광역시', 0.5448193550109863), ('국내', 0.5182844996452332), ('대구광역시', 0.5025010108947754), ('서울특별시', 0.50123131275177), ('교육인', 0.48795831203460693), ('경상북도', 0.48792165517807007)]


In [ ]:
model_result2=model2.wv.most_similar("대한민국")
print(model_result2)

[('한국', 0.6806833744049072), ('김정훈', 0.6688888669013977), ('정미경', 0.6647691130638123), ('탁구선수', 0.6479456424713135), ('최은경', 0.6457558870315552), ('이현진', 0.6324021220207214), ('이현욱', 0.6303592920303345), ('박신영', 0.6282212734222412), ('장복심', 0.6275299191474915), ('장용석', 0.6274713277816772)]


In [ ]:
model_result3=model.wv.most_similar("고양이")
print(model_result3)

[('강아지', 0.7985575795173645), ('거위', 0.7776944637298584), ('토끼', 0.7718325257301331), ('애완동물', 0.7614423632621765), ('애완견', 0.7593047618865967), ('울음소리', 0.7482358813285828), ('거북이', 0.7399520874023438), ('개구리', 0.7254258990287781), ('사냥개', 0.7131367921829224), ('생쥐', 0.6942580342292786)]


In [ ]:
model_result4=model2.wv.most_similar("고양이")
print(model_result4)

[('거위', 0.7450876235961914), ('애완견', 0.7371352314949036), ('카멜레온', 0.7318968772888184), ('생쥐', 0.7276090383529663), ('들쥐', 0.7273074984550476), ('햄스터', 0.7257925868034363), ('토끼', 0.7210389375686646), ('더치스', 0.7160189747810364), ('앵무새', 0.7088590264320374), ('바케네코', 0.7079804539680481)]


# 사전 훈련된 워드 임베딩 사용

# 1. 영어 
구글은 사전 훈련된 3백만 개의 Word2Vec 단어 벡터들을 제공합니다. 각 임베딩 벡터의 차원은 300입니다. gensim을 통해서 이 모델을 불러오는 건 매우 간단합니다. 이 모델을 다운로드하고 파일 경로를 기재하면 됩니다.

In [ ]:
import gensim

# 구글의 사전 훈련된 Word2Vec 모델을 로드합니다.
model = gensim.models.KeyedVectors.load_word2vec_format('A:\\chchdata\\data\\GoogleNews-vectors-negative300.bin\\GoogleNews-vectors-negative300.bin', binary=True) 
print(model.vectors.shape) # 모델의 크기 확인

(3000000, 300)


#### Load a word2vec model stored in the C *text* format.  
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)
#### Load a word2vec model stored in the C *binary* format.  
wv_from_bin = KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)

In [ ]:
print (model.similarity('this', 'is')) # 두 단어의 유사도 계산하기
print (model.similarity('post', 'book'))

0.40797037
0.057204388


In [ ]:
print(model['book']) # 단어 'book'의 벡터 출력

[ 0.11279297 -0.02612305 -0.04492188  0.06982422  0.140625    0.03039551
 -0.04370117  0.24511719  0.08740234 -0.05053711  0.23144531 -0.07470703
  0.21875     0.03466797 -0.14550781  0.05761719  0.00671387 -0.00701904
  0.13183594 -0.25390625  0.14355469 -0.140625   -0.03564453 -0.21289062
 -0.24804688  0.04980469 -0.09082031  0.14453125  0.05712891 -0.10400391
 -0.19628906 -0.20507812 -0.27539062  0.03063965  0.20117188  0.17382812
  0.09130859 -0.10107422  0.22851562 -0.04077148  0.02709961 -0.00106049
  0.02709961  0.34179688 -0.13183594 -0.078125    0.02197266 -0.18847656
 -0.17480469 -0.05566406 -0.20898438  0.04858398 -0.07617188 -0.15625
 -0.05419922  0.01672363 -0.02722168 -0.11132812 -0.03588867 -0.18359375
  0.28710938  0.01757812  0.02185059 -0.05664062 -0.01251221  0.01708984
 -0.21777344 -0.06787109  0.04711914 -0.00668335  0.08544922 -0.02209473
  0.31835938  0.01794434 -0.02246094 -0.03051758 -0.09570312  0.24414062
  0.20507812  0.05419922  0.29101562  0.03637695  0.04

# 2. 한국어

In [ ]:
!pip uninstall -y gensim

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Successfully uninstalled gensim-4.1.2


In [ ]:
!pip install --upgrade --user gensim==3.8.3

  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)
  Using cached Cython-0.29.14-cp38-cp38-win_amd64.whl (1.7 MB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23
  Attempting uninstall: gensim
    Found existing installation: gensim 3.8.1
    Uninstalling gensim-3.8.1:
      Successfully uninstalled gensim-3.8.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip list

Package               Version
--------------------- -------------------
argon2-cffi           20.1.0
async-generator       1.10
attrs                 21.2.0
backcall              0.2.0
beautifulsoup4        4.6.0
bleach                4.0.0
blis                  0.7.4
catalogue             2.0.6
certifi               2021.5.30
cffi                  1.14.6
charset-normalizer    2.0.6
click                 7.1.2
colorama              0.4.4
cycler                0.10.0
cymem                 2.0.5
Cython                0.29.14
decorator             5.0.9
defusedxml            0.7.1
en-core-web-sm        3.1.0
entrypoints           0.3
gensim                3.8.3
idna                  3.2
ipykernel             5.3.4
ipython               7.22.0
ipython-genutils      0.2.0
ipywidgets            7.6.3
jedi                  0.17.0
Jinja2                3.0.1
joblib                1.0.1
JPype1                1.1.2
jsonschema            3.2.0
jupyter               1.0.0
jupyter-client        6.1

In [ ]:
import gensim
from gensim.models import KeyedVectors
model = gensim.models.Word2Vec.load('A:\\chchdata\\data\\ko\\ko.bin')

In [ ]:
result=model.wv.most_similar("강아지")
print(result)

[('고양이', 0.7290453314781189), ('거위', 0.7185634970664978), ('토끼', 0.7056223750114441), ('멧돼지', 0.6950401067733765), ('엄마', 0.693433403968811), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.675035297870636), ('아빠', 0.6729634404182434), ('목걸이', 0.6512461304664612)]


In [ ]:
result=model.wv.most_similar("인공지능")
print(result)

[('컴퓨팅', 0.6520194411277771), ('가상현실', 0.6393702030181885), ('심리학', 0.63037109375), ('모델링', 0.625065267086029), ('신경망', 0.6200423836708069), ('로봇', 0.6109743118286133), ('시뮬레이션', 0.6101070642471313), ('지능', 0.6092982888221741), ('기술', 0.6087721586227417), ('기술인', 0.5957076549530029)]


In [ ]:
model.wv.save_word2vec_format('./ko_w2v') # 모델 저장
# loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드